In [ ]:
!mkdir save

In [ ]:
import joblib
from glob import glob

import sys
sys.path.append('..')
from module.utils import return_best_save_fnames, result_from_models

## Train

Train 20 times and select best 10 models, and then average the results.

```shell
for i in {1..20}
do
    python ../module/models/gru.py --submit=True --num_workers=4 --save_fname=save/gru.pt --seed=$i
    python ../module/models/gnn.py --submit=True --num_workers=4 --save_fname=save/gnn.pt --seed=$i
    python ../module/models/grun.py --submit=True --num_workers=4 --save_fname=save/grun.pt --seed=$i
    python ../module/models/grun.py --submit=True --num_workers=4 --all=True --save_fname=save/grun-all.pt --seed=$i
    python ../module/models/mlp.py --submit=True --num_workers=4 --augmentation=True --save_fname=save/mlp-augmentation.pt --seed=$i
done
```

In [ ]:
gru_save_fnames = glob('save/gru.pt*')
gnn_save_fnames = glob('save/gnn.pt*')
grun_save_fnames = glob('save/grun.pt*')
grun_all_save_fnames = glob('save/grun-all.pt*')
mlp_aug_save_fnames = glob('save/mlp-augmentation.pt*')

In [ ]:
def return_logit(saves, **kwargs):
    top_saves = return_best_save_fnames(saves, topn=10, submit=True, kind='val', **kwargs)
    return result_from_models(top_saves, submit=True, kind='leader')

In [ ]:
gru_logit = return_logit(gru_save_fnames)
gnn_logit = return_logit(gnn_save_fnames)
grun_logit = return_logit(grun_save_fnames)
grun_all_logit = return_logit(grun_all_save_fnames)
mlp_aug_logit = return_logit(mlp_aug_save_fnames)

In [ ]:
!mkdir -p logits/leader

In [ ]:
logits = [('gru', gru_logit), ('gnn', gnn_logit), ('grun', grun_logit), ('grun_all', grun_all_logit), ('mlp', mlp_aug_logit)]
joblib.dump(logits, 'logits/leader/saved_logits')